# **Graficación**
---

En este notebook se grafica el desarrollo que tuvo el entrenamiento al extraerlo de la bitácora que se generó y guardó en el notebook Entrenamiento. De esta información se visualiza la función de certeza en la generación de la máscara y la caja de las instancias, además se produce la función de error de los valores de la red entre iteración, esta métrica ayuda en el análisis de cómo estuvo aprendiendo el modelo.

# **Importar bitácora**
---
Se guardan los datos del entranmiento en una lista por si esta información se almacena en distintos archivos

In [ ]:
logFiles = [
            '/content/drive/MyDrive/PT/YOLACT_output/v10/yolact_facemask_0_0.log'
            ]

# **Elaboración de cada gráfica**
---
Código que grafica la función que se le pasa, estos valores son:


*   Calificación mAP para generar la máscara de la instancia
*   Calificación mAP para generar la caja de la instancia
*   Función de costo

Se siguió la implementación de la clase [LogVisualizer](https://github.com/dbolya/yolact/blob/master/utils/logger.py) que viene incluida en [Yolact](https://github.com/dbolya/yolact). Por errores al ejecutar esta clase de la forma que el autor [recomienda](https://github.com/dbolya/yolact/issues/78#issuecomment-582152293) con las modificaciones hechas para personalizar la visualización, es que se puso el código completo en una celda.



In [ ]:
import os
import json
from typing import Union
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from collections import deque
import math

class MovingAverage():
    """ Keeps an average window of the specified number of items. """

    def __init__(self, max_window_size=1000):
        self.max_window_size = max_window_size
        self.reset()

    def add(self, elem):
        """ Adds an element to the window, removing the earliest element if necessary. """
        if not math.isfinite(elem):
            print('Warning: Moving average ignored a value of %f' % elem)
            return
        
        self.window.append(elem)
        self.sum += elem

        if len(self.window) > self.max_window_size:
            self.sum -= self.window.popleft()
    
    def append(self, elem):
        """ Same as add just more pythonic. """
        self.add(elem)

    def reset(self):
        """ Resets the MovingAverage to its initial state. """
        self.window = deque()
        self.sum = 0

    def get_avg(self):
        """ Returns the average of the elements in the window. """
        return self.sum / max(len(self.window), 1)
    
    def __len__(self):
        return len(self.window)

class LogEntry():
    """ A class that allows you to navigate a dictonary using x.a.b[2].c, etc. """

    def __init__(self, entry:Union[dict, list]):
        self._ = entry

    def __getattr__(self, name):
        if name == '_':
            return self.__dict__['_']

        res = self.__dict__['_'][name]

        if type(res) == dict or type(res) == list:
            return LogEntry(res)
        else:
            return res
    
    def __getitem__(self, name):
        return self.__getattr__(name)

    def __len__(self):
        return len(self.__dict__['_'])

class LogVisualizer():

    COLORS = [
        'xkcd:azure',
        'xkcd:coral',
        'xkcd:turquoise',
        'xkcd:orchid',
        'xkcd:orange',
        'xkcd:blue',
        'xkcd:red',
        'xkcd:teal',
        'xkcd:magenta',
        'xkcd:orangered'
    ]

    def __init__(self):
        self.logs = []
        self.total_logs = []
        self.log_names = []
    
    def _decode(self, query:str) -> list:
        path, select = (query.split(';') + [''])[:2]
        
        if select.strip() == '':
            select = lambda x, s: True
        else:
            select = eval('lambda x, s: ' + select)

        if path.strip() == '':
            path = lambda x, s: x
        else:
            path = eval('lambda x, s: ' + path)
        
        return path, select

    def _follow(self, entry:LogEntry, query:list):
        path, select = query

        try:
            if select(entry, entry._s):
                res = path(entry, entry._s)

                if type(res) == LogEntry:
                    return res.__dict__['_']
                else:
                    return res
            else:
                return None
        except (KeyError, IndexError):
            return None

    def _color(self, idx:int):
        return self.COLORS[idx % len(self.COLORS)]

    def add(self, path:str, session:Union[int,list]=None):
        """ Add a log file to the list of logs being considered. """

        log = defaultdict(lambda: [])
        total_log = []

        if not os.path.exists(path):
            print(path + ' doesn\'t exist!')
            return

        session_idx = 0
        ignoring = True
        
        def valid(idx):
            if session is None:
                return True
            elif type(session) == int:
                return (idx == session)
            else:
                return idx in session

        with open(path, 'r') as f:
            for line in f:
                line = line.strip()
                if len(line) > 0:
                    js = json.loads(line)
                    
                    _type = js['type']
                    if _type == 'session':
                        session_idx = js['session']
                        ignoring = not valid(session_idx)

                    if not ignoring:
                        ljs = LogEntry(js)
                        if _type == 'session':
                            js['_s'] = ljs
                        else:
                            js['_s'] =log['session'][-1]
                        log[_type].append(ljs)
                        total_log.append(ljs)
        
        name = os.path.basename(path)
        if session is not None:
            name += ' (Session %s)' % session

        self.logs.append(log)
        self.total_logs.append(total_log)
        self.log_names.append(name)


    def plot(self, entry_type:str, x:str, y:str, smoothness:int=10):
        """ Plot sequential log data. """

        query_x = self._decode(x)
        query_y = self._decode(y)

        for idx, (log, name) in enumerate(zip(self.logs, self.log_names)):
            log = log[entry_type]

            if smoothness > 1:
                avg = MovingAverage(smoothness)

            _x = []
            _y = []

            for datum in log:
                val_x = self._follow(datum, query_x)
                val_y = self._follow(datum, query_y)

                if val_x is not None and val_y is not None:
                    if smoothness > 1:
                        avg.append(val_y)
                        val_y = avg.get_avg()

                        if len(avg) < smoothness // 10:
                            continue
                        
                    _x.append(val_x)
                    _y.append(val_y)
            
            expNum=['modelo_cubrebocas']
            plt.plot(_x, _y, color=self._color(idx), linewidth=1.5, label=expNum[idx])
        
        if entry_type.__eq__('val'):
            plt.title('Nivel de certeza (mAP) mask' if y.__eq__('x.data.mask["all"]') else 'Nivel de certeza (mAP) bbox', fontsize=15)
            plt.xlabel('Epocs', fontsize=14)
            plt.ylabel('Porcentaje', fontsize=14)
        elif entry_type.__eq__('train'):
            plt.title('Función de costo', fontsize=15)
            plt.xlabel('Iteraciones', fontsize=14)
            plt.ylabel('Costo', fontsize=14)
        plt.legend()
        plt.grid(linestyle=':', linewidth=1.0)
        plt.show()

if __name__ == '__main__':

    vis = LogVisualizer()
    plt.rcParams['figure.figsize'] = [4, 4] 

    for logFile in logFiles:
      vis.add(logFile)
    
    vis.plot('val', 'x.data.epoch', 'x.data.mask["all"]', smoothness=10)
    vis.plot('val', 'x.data.epoch', 'x.data.box["all"]', smoothness=10)
    vis.plot('train', 'x.data.iter', 'x.data.loss.T', smoothness=1000)
